In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

In [4]:
avisos.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_uppercase,pide_hombre,pide_mujer,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR,True,False,False,...,True,False,False,True,False,False,False,False,False,False
1,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,True,False,False,...,False,True,False,True,False,False,False,False,False,False
2,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,False,False,False,True,False,False,False,False,False,False
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [5]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion,dia,mes,semana,diadelasemana,hora
0,1112257047,NM5M,2018-01-15 16:22:34,15,1,3,Lunes,16
1,1111920714,NM5M,2018-02-06 09:04:50,6,2,6,Martes,9
2,1112346945,NM5M,2018-02-22 09:04:47,22,2,8,Jueves,9
3,1112345547,NM5M,2018-02-22 09:04:59,22,2,8,Jueves,9
4,1112237522,5awk,2018-01-25 18:55:03,25,1,4,Jueves,18


In [6]:
postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,Graduado,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,Graduado,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [7]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [8]:
prediccion['sepostulo']=0
prediccion.drop({'idaviso','idpostulante'},axis=1, inplace=True)
prediccion.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [9]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion1.csv",index=False)

In [10]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [11]:
postulaciones2=postulaciones;
postulaciones2['sepostulo']=1
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')
postulaciones2.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,-,-
1,1111920714,NM5M,1,FEM,47.0,-,-,-,-,Graduado,-,-
2,1112346945,NM5M,1,FEM,47.0,-,-,-,-,Graduado,-,-
3,1112345547,NM5M,1,FEM,47.0,-,-,-,-,Graduado,-,-
4,1112237522,5awk,1,FEM,55.0,-,-,-,-,Graduado,Graduado,Graduado


In [12]:
postulaciones3= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/2.csv")
postulaciones3.head()

,idaviso,idpostulante,sepostulo
0,1112257047,pzjAxkp,0
1,1111920714,E5wmpz,0
2,1112346945,5mdDbaz,0
3,1112345547,bOj9Dr0,0
4,1112237522,ElMpN0,0


In [13]:
postulaciones3=pd.merge(postulaciones3,postulantes,on='idpostulante')
postulaciones3.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,1112257047,pzjAxkp,0,FEM,32.0,-,-,-,-,-,-,En Curso
1,1112269753,pzjAxkp,0,FEM,32.0,-,-,-,-,-,-,En Curso
2,1112322432,pzjAxkp,0,FEM,32.0,-,-,-,-,-,-,En Curso
3,1112265973,pzjAxkp,0,FEM,32.0,-,-,-,-,-,-,En Curso
4,1112302772,pzjAxkp,0,FEM,32.0,-,-,-,-,-,-,En Curso


In [14]:
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')
postulaciones3=pd.merge(postulaciones3,avisos,on='idaviso')

In [15]:
postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [16]:
postulaciones3.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,pzjAxkp,0,FEM,32.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,owYMbeW,0,MASC,36.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,Rz6GZEq,0,FEM,24.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,dY98kj0,0,FEM,57.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,2zPxZB0,0,MASC,18.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [17]:
postulaciones2=postulaciones2.append(postulaciones3, ignore_index=True)
postulaciones2.sample(10).head(10)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
11768567,1112237727,1zL3pj,0,MASC,46.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,False
2376021,1112285718,ekOOYmr,1,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,False
2431302,1112417585,1Qd4VjR,1,FEM,20.0,-,-,Graduado,-,Graduado,...,True,False,False,False,False,False,False,False,False,False
761678,1112308313,JBrP4RW,1,MASC,34.0,-,Graduado,-,-,Graduado,...,False,False,False,False,False,True,False,False,True,False
2165519,1112440565,5mrMoVw,1,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,False
5622914,1112357605,ZQZJJb,1,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,False
5434890,1112329637,lD2dKZM,1,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,False
3293115,1112361820,88GLlL,1,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,False
7505463,1112353152,lDXrOvj,0,MASC,35.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,True
7012885,1112263547,NJ0aQB,0,FEM,33.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,False


In [18]:
postulaciones3=0
postulaciones2.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [19]:
zonas=avisos['nombre_zona'].value_counts().index
postulaciones2['nombre_zona']=postulaciones2['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
postulaciones2['tipo_de_trabajo']=postulaciones2['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
postulaciones2['nivel_laboral']=postulaciones2['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
postulaciones2['nombre_area']=postulaciones2['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
postulaciones2['Doctorado']=postulaciones2['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
postulaciones2['Master']=postulaciones2['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
postulaciones2['Otro']=postulaciones2['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
postulaciones2['Posgrado']=postulaciones2['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
postulaciones2['Secundario']=postulaciones2['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
postulaciones2['Terciario/Técnico']=postulaciones2['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
postulaciones2['Universitario']=postulaciones2['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
postulaciones2['sexo']=postulaciones2['sexo'].replace(sexo, range(len(sexo)))
postulaciones2.dtypes

idaviso                        int64
idpostulante                  object
sepostulo                      int64
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [20]:
train, test=train_test_split(postulaciones2,test_size=0.10)

In [21]:
print("Train: ",len(train),"Test: ",len(test))

Train:  11098670 Test:  1233186


In [22]:
features=list(postulaciones2.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')
features

['sexo',
 'edad',
 'Doctorado',
 'Master',
 'Otro',
 'Posgrado',
 'Secundario',
 'Terciario/Técnico',
 'Universitario',
 'nombre_zona',
 'tipo_de_trabajo',
 'nivel_laboral',
 'nombre_area',
 'titulo_uppercase',
 'pide_hombre',
 'pide_mujer',
 'ingles',
 'experiencia',
 'paquete_office',
 'liderazgo',
 'secundario',
 'viajar',
 'empresa_importante',
 'capacitacion',
 'remuneracion_pretendida',
 'graduados',
 'estudiantes',
 'conocimientos_especificos',
 'buen_ambiente',
 'discriminacion']

In [23]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [24]:
c=DecisionTreeClassifier(min_samples_split=100)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/arbol.pkl") 
dt=c.fit(x_train,y_train)

In [25]:
y_pred=c.predict(x_test)
score=accuracy_score(y_test,y_pred)*100
print(score)

67.39924066604713


In [ ]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/arbol_ciego.pkl")

In [26]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
55340,5133,1111520507,1oaPN,MASC,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,False
30299,87759,1112466925,EzEzpRo,MASC,21.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,False
59890,37044,1112412845,JBrkXQJ,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,False
39230,83717,1112466410,0zP4Kor,MASC,26.0,-,-,-,-,Graduado,...,True,False,True,False,False,False,False,False,False,False
80936,40252,1112418747,5LBWpM,MASC,58.0,-,-,-,-,-,...,False,False,True,False,False,True,False,False,False,False
14745,64018,1112457790,LN8528j,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,True,False,False,False
26068,25270,1112334788,Z2Y6kb,FEM,41.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,True,False
43477,24943,1112325449,6Ov9oM,FEM,29.0,-,-,-,-,-,...,True,False,False,False,False,False,False,True,False,True
40238,40098,1112418587,8QKOaW,FEM,33.0,-,-,Graduado,-,Graduado,...,True,False,True,False,False,False,False,False,True,False
19705,79486,1112464706,0zPVrDr,FEM,23.0,-,-,-,-,Graduado,...,True,False,True,False,False,False,False,False,False,False


In [27]:
zonas=avisos['nombre_zona'].value_counts().index
prediccion['nombre_zona']=prediccion['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
prediccion['tipo_de_trabajo']=prediccion['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
prediccion['nivel_laboral']=prediccion['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
prediccion['nombre_area']=prediccion['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
prediccion['Doctorado']=prediccion['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
prediccion['Master']=prediccion['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
prediccion['Otro']=prediccion['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
prediccion['Posgrado']=prediccion['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
prediccion['Secundario']=prediccion['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
prediccion['Terciario/Técnico']=prediccion['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
prediccion['Universitario']=prediccion['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
prediccion['sexo']=prediccion['sexo'].replace(sexo, range(len(sexo)))
prediccion.dtypes

id                             int64
idaviso                        int64
idpostulante                  object
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [28]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [29]:
prediccion['sepostulo']=y_final

In [30]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,sepostulo
0,0,739260,6M9ZQR,0,42.0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,1
1,1,739260,6v1xdL,1,30.0,0,0,1,0,1,...,False,False,False,False,False,False,False,False,False,0
2,2,739260,ezRKm9,0,36.0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,1
3,3,758580,1Q35ej,1,68.0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0
4,4,758580,EAN4J6,0,32.0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,0


In [31]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0


In [32]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion2.csv",index=False)

In [33]:
prediccion.sepostulo.mean()

0.54247